# CUDA を使って画像に3DLUTをかける

## 3DLUT の補間処理の元ネタ

FUJIFILM社の「LOGとLUTの基礎講座」資料 P.38 のアルゴリズムを使用する。

| |
|:-----:|
|![figure1](./figure/3dlut_interpolation.jpg)|


リンク：[LOGとLUTの基礎講座](http://fujifilm.jp/business/broadcastcinema/solution/color_management/is-mini/promotion/pack/pdf/news/20150122_jppa.pdf)

## Python向けに補間処理の数式を考える。

![figure1](./figure/sekkei.jpg)

8頂点で囲われた立方体を上図を参考に8領域に分ける。それぞれの領域の体積を$V_0 \sim V_7$ と置くと、以下の式で算出できる。

\begin{align}
V_0 & = R_0 \cdot G_0 \cdot B_0 \\
V_1 & = R_0 \cdot G_0 \cdot B_1 \\
V_2 & = R_0 \cdot G_1 \cdot B_0 \\
V_3 & = R_0 \cdot G_1 \cdot B_1 \\
V_4 & = R_1 \cdot G_0 \cdot B_0 \\
V_5 & = R_1 \cdot G_0 \cdot B_1 \\
V_6 & = R_1 \cdot G_1 \cdot B_0 \\
V_7 & = R_1 \cdot G_1 \cdot B_1 \\
\end{align}

ここで $V'_i = 1 - V_i$ と置くと、3DLUTの出力は以下の式で求められる。

$$
\begin{align}
R_{out} & = V'_0 \cdot L_0[R] + V'_1 \cdot L_1[R] + V'_2 \cdot L_2[R] + V'_3 \cdot L_3[R] \\
& + V'_4 \cdot L_4[R] + V'_5 \cdot L_5[R] + V'_6 \cdot L_6[R] + V'_7 \cdot L_7[R] \\
G_{out} & = V'_0 \cdot L_0[G] + V'_1 \cdot L_1[G] + V'_2 \cdot L_2[G] + V'_3 \cdot L_3[G] \\ 
& + V'_4 \cdot L_4[G] + V'_5 \cdot L_5[G] + V'_6 \cdot L_6[G] + V'_7 \cdot L_7[G] \\
B_{out} & = V'_0 \cdot L_0[B] + V'_1 \cdot L_1[B] + V'_2 \cdot L_2[B] + V'_3 \cdot L_3[B] \\
& + V'_4 \cdot L_4[B] + V'_5 \cdot L_5[B] + V'_6 \cdot L_6[B] + V'_7 \cdot L_7[B]
\end{align}
$$

## Grid数を考慮した数式を立ててみよう

グリッド数を$N$とする。$N$ は整数 $m$ によって以下の式で求められる。<br>
ひとまずは、m=4, 5, 6 あたりをターゲットとする。

$$
N = 2^m + 1
$$

入力画像を $D(w, h, c)$ とする。$w, h$ は空間方向の座標を $c$ は 色方向の座標を意味する。<br>
入力画像は$N$に合わせて正規化する。正規化後の入力画像を $D'$ とする。

$$
D' = \frac{D}{{\rm max}(D)} \cdot N
$$

さて、この時 $D'$ が参照するグリッドのインデックス $i, j, k$ は以下となる。<br>
※ $D=N$ つまり最大値の時に、本当に端っこのグリッドに当たるか不安。要確認。

$$
i = \lfloor D' \rfloor
$$

In [17]:
# 上のコードを関数化
# ---------------------------------
import sys
import imp
sys.path.append('./code')
import pycuda_3dlut as p3
imp.reload(p3)
p3.mono_conv_matrix()